In [53]:
# Updated 14Aug2017
%load_ext edx_jupyter

The edx_jupyter extension is already loaded. To reload it, use:
  %reload_ext edx_jupyter


In [54]:
%reload_ext edx_jupyter

In [55]:
%vertica SELECT 'Connected to Vertica'

,?column?
0,Connected to Vertica


In [ ]:
%vertica SET TIME ZONE to UTC;

In [62]:
days_since_completion = 14
days_before_ver_deadline = 5

## Experiment: Table of nonverified passing learners (self-paced without differentiated assessments), last 14 days, future verification date not within the next 5 days.

In [63]:
%%vertica (days_since_completion=days_since_completion,days_before_ver_deadline=days_before_ver_deadline)
-- This query will still need to exclude the list of excluded courses which are currently in csv file
DROP TABLE IF EXISTS non_verified_passing_users;
CREATE local TEMPORARY TABLE non_verified_passing_users
ON COMMIT PRESERVE ROWS AS
SELECT
    distinct
    a.user_id,
    a.course_id,
    b.current_enrollment_mode,
    a.letter_grade,
    a.passed_timestamp,
    c.course_verification_end_date,
    c.course_seat_price,
    c.pacing_type
FROM business_intelligence.course_completion_user a
JOIN production.d_user_course b
    ON a.passed_timestamp is not null
    AND a.passed_timestamp >= (CURRENT_DATE - :days_since_completion)
    AND b.current_enrollment_mode IN ('honor','audit')
    AND a.user_id = b.user_id
    AND a.course_id = b.course_id
JOIN business_intelligence.course_master c
    ON a.course_id = c.course_id
    AND c.course_verification_end_date >= (CURRENT_DATE - :days_before_ver_deadline)
    AND c.pacing_type = 'self_paced'

In [64]:
%%vertica
select count(user_id) from non_verified_passing_users

,count
0,8249


In [65]:
%%vertica
select
    pacing_type,
    count(user_id) 
from non_verified_passing_users
group by pacing_type

,pacing_type,count
0,self_paced,8249


## Historical: Table of all nonverified passing learners, since 2 weeks ago, with future verification date

In [7]:
%%vertica
DROP TABLE IF EXISTS non_verified_passing_users;
CREATE local TEMPORARY TABLE non_verified_passing_users
ON COMMIT PRESERVE ROWS AS
SELECT
    distinct
    a.user_id,
    a.course_id,
    b.current_enrollment_mode,
    a.letter_grade,
    a.passed_timestamp,
    c.course_verification_end_date,
    c.course_seat_price,
    c.pacing_type
FROM business_intelligence.course_completion_user a
JOIN production.d_user_course b
    ON a.passed_timestamp is not null
    AND a.passed_timestamp >= '2017-07-14'
    AND b.current_enrollment_mode IN ('honor','audit')
    AND a.user_id = b.user_id
    AND a.course_id = b.course_id
JOIN business_intelligence.course_master c
    ON a.course_id = c.course_id
    AND c.course_verification_end_date >= CURRENT_DATE()

,pacing_type,count


In [8]:
%%vertica
select count(user_id) from non_verified_passing_users

,count
0,8274


In [9]:
%%vertica
select
    pacing_type,
    count(user_id) 
from non_verified_passing_users
group by pacing_type

,pacing_type,count
0,instructor_paced,924
1,self_paced,7350


In [10]:
%%vertica
select
course_id,
current_enrollment_mode,
course_seat_price,
count(distinct user_id)
from
non_verified_passing_users
group by 1,2,3
order by count desc

,course_id,current_enrollment_mode,course_seat_price,count
0,course-v1:Microsoft+DAT101x+3T2017,audit,99.0,433
1,course-v1:CatalystX+ILDIV1x+2T2017,audit,25.0,303
2,course-v1:Microsoft+DAT208x+3T2017,audit,99.0,249
3,course-v1:Microsoft+DEV276x+2T2017,audit,99.0,249
4,course-v1:AdelaideX+Project101x+1T2017,audit,50.0,216
5,course-v1:RITx+CYBER503x+2T2017,audit,150.0,187
6,course-v1:Microsoft+DEV210x+3T2017,audit,99.0,182
7,course-v1:UPValenciaX+PPT101x+2T2017,audit,50.0,180
8,course-v1:Microsoft+DAT206x+3T2017,audit,99.0,178
9,course-v1:Microsoft+DAT201x+3T2017,audit,99.0,156


## Historical: All non-verified passing learners, since beginning of time

In [11]:
%%vertica
DROP TABLE IF EXISTS non_verified_passing_users_all;
CREATE local TEMPORARY TABLE non_verified_passing_users_all
ON COMMIT PRESERVE ROWS AS
SELECT
    distinct a.user_id,
    a.course_id,
    a.letter_grade,
    a.passed_timestamp,
    b.current_enrollment_mode,
    c.course_verification_end_date,
    c.course_seat_price,
    c.pacing_type,
    b.first_verified_enrollment_time is NOT NULL AS eventually_verified_after_passing
    FROM business_intelligence.course_completion_user a
JOIN production.d_user_course b
    ON a.passed_timestamp is not null
    AND a.user_id = b.user_id
    AND a.course_id = b.course_id
    AND 
    (
        -- User verified after passing
        (
            b.first_verified_enrollment_time is NOT NULL
            AND b.first_verified_enrollment_time > a.passed_timestamp
        ) OR
        -- User passed, but did not verify
        b.first_verified_enrollment_time is NULL
    )
JOIN business_intelligence.course_master c
    ON a.course_id = c.course_id
    AND c.pacing_type is NOT NULL

,course_id,current_enrollment_mode,course_seat_price,count


In [12]:
%%vertica -> nvpu
SELECT
    pacing_type,
    eventually_verified_after_passing,
    count(user_id)
FROM
    non_verified_passing_users_all
GROUP BY pacing_type, eventually_verified_after_passing

In [13]:
nvpu.set_index('pacing_type', 'eventually_verified_after_passing')

,eventually_verified_after_passing,count
pacing_type,,
instructor_paced,False,722711
self_paced,False,468432
self_paced,True,53451
instructor_paced,True,20437


## Historical: All non-verified passing learners, since beginning of time, with future verification date

In [14]:
%%vertica
DROP TABLE IF EXISTS non_verified_passing_users_all_future_v_date;
CREATE local TEMPORARY TABLE non_verified_passing_users_all_future_v_date
ON COMMIT PRESERVE ROWS AS
SELECT
    distinct a.user_id,
    a.course_id,
    a.letter_grade,
    a.passed_timestamp,
    b.current_enrollment_mode,
    c.course_verification_end_date,
    c.course_seat_price,
    c.pacing_type,
    b.first_verified_enrollment_time is NOT NULL AS eventually_verified_after_passing
FROM business_intelligence.course_completion_user a
JOIN production.d_user_course b
    ON a.passed_timestamp is not null
    AND a.user_id = b.user_id
    AND a.course_id = b.course_id
    AND 
    (
        -- User verified after passing
        (
            b.first_verified_enrollment_time is NOT NULL
            AND b.first_verified_enrollment_time > a.passed_timestamp
        ) OR
        -- User passed, but did not verify
        b.first_verified_enrollment_time is NULL
    )
JOIN business_intelligence.course_master c
    ON a.course_id = c.course_id
    AND c.pacing_type is NOT NULL
    AND c.course_verification_end_date >= CURRENT_DATE()

,pacing_type,eventually_verified_after_passing,count


In [15]:
%%vertica -> nvpu
SELECT
    pacing_type,
    eventually_verified_after_passing,
    count(user_id)
FROM
    non_verified_passing_users_all_future_v_date
GROUP BY pacing_type, eventually_verified_after_passing

In [16]:
nvpu.set_index('pacing_type', 'eventually_verified_after_passing')

,eventually_verified_after_passing,count
pacing_type,,
instructor_paced,False,2572
instructor_paced,True,200
self_paced,False,80511
self_paced,True,9188
